In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pandas as pd
import math

In [2]:
data = pd.read_csv('../1.data-preparation/new-data/data_Y1.csv')
data.describe()

,MemberID,ClaimsTruncated,TARGET,trainset,age_05,age_15,age_25,age_35,age_45,age_55,...,drugCount_max,drugCount_min,drugCount_ave,drugcount_months,labCount_max,labCount_min,labCount_ave,labcount_months,labNull,drugNull
count,7.603700e+04,76037.000000,76037.000000,76037.0,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,...,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000
mean,4.994795e+07,0.052225,0.467102,1.0,0.085866,0.090824,0.059405,0.096098,0.132935,0.112366,...,2.175033,0.968765,1.511600,3.703697,3.758749,2.178216,2.885066,1.580296,0.299999,0.344569
std,2.890149e+07,0.222481,1.612204,0.0,0.280168,0.287361,0.236383,0.294727,0.339507,0.315819,...,2.243436,1.046544,1.536634,4.089940,3.434990,2.533432,2.696510,1.814171,0.458260,0.475231
min,2.100000e+02,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.483922e+07,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.990758e+07,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,1.000000,1.200000,2.000000,3.000000,1.000000,2.600000,1.000000,0.000000,0.000000
75%,7.510115e+07,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.285714,7.000000,6.000000,3.000000,4.800000,2.000000,1.000000,1.000000
max,9.999863e+07,1.000000,15.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,7.000000,7.000000,7.000000,12.000000,10.000000,10.000000,10.000000,12.000000,1.000000,1.000000


In [3]:
data.drop("MemberID", axis = 1, inplace = True)
data.drop("Year", axis = 1, inplace = True)

y = data['TARGET']
X = data.loc[:, data.columns != 'TARGET']
X.head()

,ClaimsTruncated,trainset,age_05,age_15,age_25,age_35,age_45,age_55,age_65,age_75,...,drugCount_max,drugCount_min,drugCount_ave,drugcount_months,labCount_max,labCount_min,labCount_ave,labcount_months,labNull,drugNull
0,0,1,0,0,0,0,0,1,0,0,...,0,0,0.00000,0,10,10,10.000000,1,0,1
1,0,1,0,0,0,0,0,0,0,1,...,5,2,3.90909,11,6,1,2.750000,4,0,0
2,0,1,0,0,0,0,0,0,0,1,...,0,0,0.00000,0,6,6,6.000000,1,0,1
3,0,1,0,0,0,0,0,0,0,1,...,4,1,2.00000,6,6,1,3.333333,3,0,0
4,0,1,0,0,0,0,0,0,0,1,...,2,2,2.00000,1,5,2,2.750000,4,0,0


In [4]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [15]:
def eval(model, X_test, y_test):
    pred_y = model.predict(X_test) # your predicted y values
    # The root mean square logarithmic error
    pred_y = pred_y.clip(min=0)
    pred_y = pred_y.clip(max=15)
    pred_y = pred_y.round(0).astype(int)
    msle = np.mean( (np.log(pred_y + 1) - np.log(y_test + 1)) ** 2)
    rmsle = math.sqrt(msle)
    print ("RMSLE: %.2f" % rmsle)
    from sklearn.metrics import r2_score
    r2 = r2_score(y_test,pred_y)
    print ("R2 value: %.2f" % r2)
    from sklearn.metrics import mean_squared_log_error
    err = math.sqrt(mean_squared_log_error(y_test,pred_y))
    print ("Err value: %.2f" % err)

In [18]:
from sklearn.neural_network import MLPRegressor

model= MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=7, learning_rate='adaptive',
        learning_rate_init=0.001, max_iter=3000, momentum=0.9,
        nesterovs_momentum=True, power_t=0.5, random_state=None,
        shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
        verbose=1, warm_start=False)

# param_grid = {'hidden_layer_sizes': [i for i in range(1,15)],
#               'activation': ['relu'],
#               'solver': ['adam'],
#               'learning_rate': ['constant'],
#               'learning_rate_init': [0.001],
#               'power_t': [0.5],
#               'alpha': [0.0001],
#               'max_iter': [1000],
#               'early_stopping': [False],
#               'warm_start': [False]}
# _GS = GridSearchCV(model, param_grid=param_grid,scoring='neg_mean_squared_log_error',
#                    cv=3, verbose=True, pre_dispatch='2*n_jobs')
model.fit(X_train, y_train)
# model.fit(X_train, y_train)
eval(model, X_test, y_test)

Iteration 1, loss = 1.54726773
Iteration 2, loss = 1.29808267
Iteration 3, loss = 1.24589574
Iteration 4, loss = 1.22350692
Iteration 5, loss = 1.21213244
Iteration 6, loss = 1.20446057
Iteration 7, loss = 1.19858464
Iteration 8, loss = 1.19387302
Iteration 9, loss = 1.19024228
Iteration 10, loss = 1.18719752
Iteration 11, loss = 1.18397556
Iteration 12, loss = 1.18130838
Iteration 13, loss = 1.17925843
Iteration 14, loss = 1.17706033
Iteration 15, loss = 1.17611914
Iteration 16, loss = 1.17407149
Iteration 17, loss = 1.17344073
Iteration 18, loss = 1.17183258
Iteration 19, loss = 1.16980402
Iteration 20, loss = 1.16963944
Iteration 21, loss = 1.16681807
Iteration 22, loss = 1.16815858
Iteration 23, loss = 1.16587024
Iteration 24, loss = 1.16537908
Iteration 25, loss = 1.16448874
Iteration 26, loss = 1.16411688
Iteration 27, loss = 1.16382022
Iteration 28, loss = 1.16317270
Iteration 29, loss = 1.16200423
Iteration 30, loss = 1.16115992
Iteration 31, loss = 1.16132620
Iteration 32, los

In [14]:
_GS.best_params_, _GS.best_estimator_

({'activation': 'relu',
  'alpha': 0.0001,
  'early_stopping': False,
  'hidden_layer_sizes': 1,
  'learning_rate': 'constant',
  'learning_rate_init': 0.001,
  'max_iter': 1000,
  'power_t': 0.5,
  'solver': 'adam',
  'warm_start': False},
 MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=1, learning_rate='constant',
        learning_rate_init=0.001, max_iter=1000, momentum=0.9,
        nesterovs_momentum=True, power_t=0.5, random_state=None,
        shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
        verbose=1, warm_start=False))

In [8]:
data_submit = pd.read_csv('../1.data-preparation/new-data/data_Y2.csv')
data_submit.drop("MemberID", axis = 1, inplace = True)
data_submit.drop("Year", axis = 1, inplace = True)

y_submit = data_submit['TARGET']
X_submit = data_submit.loc[:, data_submit.columns != 'TARGET']

X_submit = scaler.transform(X_submit)

In [19]:
eval(model, X_submit, y_submit)

RMSLE: 0.54
R2 value: 0.02
Err value: 0.54


In [13]:
data = pd.read_csv('../1.data-preparation/new-data/data_Y2.csv')
pred_y = model.predict(X_submit)

submission = pd.DataFrame()
submission['MemberID'] = data['MemberID']

pred_y = pred_y.clip(min=0)
pred_y = pred_y.clip(max=15)
# pred_y = pred_y.astype(int)

submission['DIH'] = pred_y
submission.groupby(['DIH']).size()
submission.to_csv('../5.ensemble/mlp.csv',index=False)

In [14]:
submission[submission['DIH']>=7]

,MemberID,DIH
16966,17124817,8.246692


In [43]:
real = pd.read_csv('../1.data-preparation/new-data/data_Y2.csv')


In [44]:
real[real['MemberID']==17124817]

,MemberID,Year,ClaimsTruncated,TARGET,trainset,age_05,age_15,age_25,age_35,age_45,...,drugCount_max,drugCount_min,drugCount_ave,drugcount_months,labCount_max,labCount_min,labCount_ave,labcount_months,labNull,drugNull
16966,17124817,Y2,1,10.0,1,0,0,0,0,0,...,7,1,5.181818,11,5,5,5.0,1,0,0
